In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import preprocess_input

In [2]:
! pip install glob2
from glob2 import glob

  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9307 sha256=42a8ecbc7488b69329da0b31eeb4b8279e71b89627bde06b051e4c4ae08178a3
  Stored in directory: /root/.cache/pip/wheels/d7/3c/72/5300602ba1269ffce8cff5dcf7b525fee756b57455903c37ba
Successfully built glob2


In [3]:
IMAGE_SIZE=[224,224]

In [4]:
vgg = VGG16(input_shape= IMAGE_SIZE + [3],weights='imagenet',include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [5]:
for layer in vgg.layers:
    layer.trainable = False

In [6]:
folders = glob('../input/last-tank/final_tanks_data/train')

In [7]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
x = Flatten()(vgg.output)

In [8]:
from tensorflow.keras.models import Model
prediction = Dense(len(folders),activation='softmax')(x)
model = Model(inputs= vgg.input,outputs=prediction)

In [9]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
training_set = train_datagen.flow_from_directory('../input/last-tank/final_tanks_data/train',target_size=(224,224),batch_size=32,class_mode='binary')

Found 12319 images belonging to 2 classes.


In [12]:
valid_set = test_datagen.flow_from_directory('../input/last-tank/final_tanks_data/valid',target_size=(224,224),batch_size=32,class_mode='binary')

Found 3503 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(training_set,validation_data=valid_set,epochs=5,steps_per_epoch=len(training_set),validation_steps=len(valid_set))